# Turn application monitorable

### *infospace and events*

When building the SkeletonApplication we understand that for the Application to be configurable, we inherit from xdata::ActionListener, attach to and implement the callback "actionPerformed"for xdata::Event "xdaq-event:setDefaultValues". 

But it's better just to remember the principles than code patterns. Everything is a reaction to something else in a xdaq process including configuring an application. Imagine a time axis in the universe of your context, all applications can fire an event at a given point in time. While someone fires a simple mark <font color="green">toolbox::Event</font>, someone else fires a bubble <font color="green">xdata::Event</font>. The bubble type has a data container (infospace) associated with it. 

By construction, an Application shares an infospace with Executive whose handle we acquire with getApplicationInfoSpace().
We inject a parameter into it then fire itemavailable event. The framework listens to the event and matches application properties in the xml file with the parameters in application infospaces and fires "xdaq-event:setDefaultValues" when done. The monster xmlns= string in the application property refers to the name of the application infospace.


   ```
   <properties xmlns="urn:xdaq-application:bril::timesource::Application" xsi:type="soapenc:Struct">
      <AAA xsi:type="xsd:boolean">true</AAA>
   </properties>
   ```
   
We get notified only if we are a listener, hence appInfoSpace->addListener(this, "urn:xdaq-event:setDefaultValues");

Configuration is a case of sharing data using the application infospace. We can create as many infospaces as we like. Different applications (in the same context) can attach to the same infospace and coordinate timing using xdata::Event or simply exchange data. For example, Application A and B are attached to a common infospace, A will do something only if the run number is changed by B. We can also define and fire our own events etc.

Once understood the principles, monitoring is just another such usecase. 

Here's a table of frequently used events. You are free to define your own events.

   event | what happened | callback | how to listen (is=infospace) | how to fire (is=infospace)
   --- | --- | --- | ---
   urn:xdaq-event:setDefaultValues | application properties acquired | actionPerformed(xdata::Event) | is->addListener(this,"urn:xdaq-event:setDefaultValues") | not you
   eventing::api::BusReadyToPublish | the bus is ready to publish | actionPerformed(toolbox::Event) | getEventingBus(name).addActionListener | not you
   urn:xdata-event:ItemAvailableEvent | a parameter is registered | actionPerformed(xdata::Event) | is->addItemAvailableListener | is->fireItemAvailable
   urn:xdata-event:ItemGroupChangedEvent | a group of parameters changed | actionPerformed(xdata::Event) | is->addGroupChangedListener | is->fireItemGroupChanged
   urn:xdata-event:ItemChangedEvent | a parameter changed | actionPerformed(xdata::Event) | is->addItemChangedListener | is->fireItemValueChanged
   urn:xdata-event:ItemGroupRetrieveEvent | update a group of parameters | implicit | is->addGroupRetrieveListener | is->fireItemGroupRetrieve
   urn:xdata-event:ItemRetrieveEvent | update a parameter | implicit | is->addItemRetrieveListener | is->fireItemValueRetrieve
   

### *Pull a group monitorables from web page and Push another group to a collector*

We write a xdaq application adding monitoring features to our last application.

Two typical monitoring scenarios are implemented: Pull and Push mode.

In [1]:
# After having checked out bri/daq 
# Compiled packages daq/bril/fakeru, daq/bril/timesource, daq/bril/dummysource
# Go to daq/bril/dummysource
workdir = %pwd
%cd $workdir/daq/bril/dummysource

/home/zhen/work/pynb/sandbox/daq/bril/dummysource


##### write Monitorable

In [2]:
# The application is identical to dummysource::Application
# In addition, we define two monitoring infospaces: m_monInfoSpacePull, m_nonInfoSpacePush
# We inherit from TimeListener because we want to do things on a timer
#
# m_monInfoSpacePull is associated with metric m_monItemListPull which we use by ourselves checking the sanity of the application 
# m_monInfoSpacePush is associated with metric m_monItemListPush which we send to a collector who checks online our data quality
# Note: any items living in the infospace must be of xdata types
%cat include/bril/dummysource/Monitorable.h

#ifndef _bril_dummysource_Monitorable_h_
#define _bril_dummysource_Monitorable_h_
#include <string>
#include <list>
#include "xdaq/Application.h"
#include "xdata/ActionListener.h"
#include "xgi/Method.h"
#include "xgi/Output.h"
#include "xgi/exception/Exception.h"
#include "xdata/InfoSpace.h"
#include "xdata/Boolean.h"
#include "xdata/String.h"
#include "xdata/Float.h"

#include "eventing/api/Member.h"
#include "toolbox/mem/MemoryPoolFactory.h"
#include "toolbox/mem/Reference.h"
#include "b2in/nub/exception/Exception.h"

#include "toolbox/BSem.h"

/**
   A xdaq application that when configured as simulation mode simulates data on bril timing signal; as real mode, it fetches data from readout queue and publish data to bril eventing.  
Feats:
   1. Configurable
   2. Subscribe to bril eventing
   3. Generate simulated data
   4. Publish to bril eventing
   5. Fetch data from readout queue in a workloop
   6. Has web page and pull monitoring items from web page
   7. Push monitoring items

##### implement Monitorable

In [3]:
# Everything else identical to Application.cc 
# Jump to read directly regions marked by "monitoring"
#
# In constructor, we create the two infospaces for monitoring
#
# Note: with createQualifiedInfoSpace method, we can create unlimited number of infospaces 
# with the same string parameter without the risk of name clash in the same context. This is because
# the system automatically attach a globally unique id (uuid) to the name. But then when looking for a 
# specific infospace, we have to use find , or match(regex)
# 
# In m_monInfoSpacePull, we want to check urn, classname, simSource, signalTopic, ruqueueSize
#                        so we put them in this infospace and attach the metric to it
# Since m_monInfoSpacePull is for ourselves,we want to check it when we want. So, when? How about on each load of 
# the application web page. The page is defined in the "Default" method which is executed on load, so we fireItemGroupRetrieve 
# event from there.
# The monitoring metric will be updated on that call. But only if this application listens to that event. Hence, we immediately 
# subscribe to it: m_monInfoSpacePull->addGroupRetrieveListener(this);
# This is called the "Pull" mode because parameters are updated when asked for(pulled).
#
# In the same way, we define a metric and attach it to m_monInfoSpacePush
# We want to send this metric regularly to the monInfoSpacePush where is somebody else's, e.g. MonitoringCollector, problem to
# read it. So in this application we don't listen to any event attached to monInfoSpacePush
# 
# In method "Default", as explained we fire fireItemGroupRetrieve. Then dump the updated parameters to a web table.
# 
# In workloop method "reading", we update parameters: m_avgValue, m_ruqueueSize;
#
# In actionPerformed(xdata::Event& e), at configuration time, we start the timer for pushing into m_monInfoSpacePush
# 
# In the timer callback method timeExpired, we fireItemGroupChanged of the 2nd monitoring metric into m_monInfoSpacePush
# Note: it is "Changed" not "Available" because the same parameters are pushed many times.
#
%cat src/common/Monitorable.cc

#include "cgicc/CgiDefs.h"
#include "cgicc/Cgicc.h"
#include "cgicc/HTTPHTMLHeader.h"
#include "cgicc/HTMLClasses.h"
#include "xcept/tools.h"
#include "toolbox/mem/HeapAllocator.h"
#include "toolbox/mem/MemoryPoolFactory.h"
#include "toolbox/task/WorkLoop.h"
#include "toolbox/task/WorkLoopFactory.h"
#include "toolbox/task/TimerFactory.h"
#include "toolbox/TimeInterval.h"
#include "toolbox/TimeVal.h"
#include "toolbox/task/Timer.h"
#include "xdata/InfoSpaceFactory.h"
#include "b2in/nub/Method.h"
#include "bril/dummysource/Monitorable.h" 
#include "bril/dummysource/exception/Exception.h" 
#include "fake/FakeRU.h"
#include <unistd.h>

XDAQ_INSTANTIATOR_IMPL (bril::dummysource::Monitorable)

bril::dummysource::Monitorable::Monitorable(xdaq::ApplicationStub* s) throw (xdaq::exception::Exception) : xdaq::Application(s),eventing::api::Member(this),m_applock(toolbox::BSem::FULL){
  this->getEventingBus("brildata").addActionListener(this);
  xgi::bind(this, &bril::dummysource::Monitorable::Defa

##### check the pulled monitoring parameters

In [4]:
# Go to the scripts directory 
%cd $workdir/daq/bril/scripts

/home/zhen/work/pynb/sandbox/daq/bril/scripts


In [6]:
# Run LocalMonitorable or LocalMonitorable
# Identical to LocalApplication. Monitorable is in sim mode and synchronized to NB64. 
%cat $workdir/daq/bril/scripts/dummysource/LocalMonitorable.template

<?xml version="1.0" encoding="UTF-8"?>
<xc:Partition xmlns:soapenc="http://schemas.xmlsoap.org/soap/encoding/" xmlns:xc="http://xdaq.web.cern.ch/xdaq/xsd/2004/XMLConfiguration-30" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">

 <xc:Context url="http://%hostname%:%contextport%">
    <xc:Endpoint protocol="utcp" service="b2in" rmode="select" hostname="%hostname%" port="%endpoint%" network="utcp1" sndTimeout="0" rcvTimeout="0" affinity="RCV:P,SND:W,DSR:W,DSS:W" singleThread="true" publish="true"/>

    <xc:Application class="pt::utcp::Application" id="12" instance="0" network="local">
      <properties xmlns="urn:xdaq-application:pt::utcp::Application" xsi:type="soapenc:Struct">
	<maxClients xsi:type="xsd:unsignedInt">10</maxClients>
        <autoConnect xsi:type="xsd:boolean">false</autoConnect>
        <ioQueueSize xsi:type="xsd:unsignedInt">65536</ioQueueSize>
        <eventQueueSize xsi:type="xsd:unsignedInt">65536</eventQueueSize>
        <maxReceiveBuffers xsi:type="xsd:uns

In [15]:
# Generate configuration xml
!./generatexml.sh dummysource

sources:
    dummysource/template.param,dummysource/LocalDataSimulator_inzone.template
generated:
    /home/zhen/work/pynb/sandbox/daq/bril/scripts/LocalDataSimulator_inzone.xml
command per context:
   /opt/xdaq/bin/xdaq.exe -p 50000 -c /home/zhen/work/pynb/sandbox/daq/bril/scripts/LocalDataSimulator_inzone.xml -z bril

sources:
    dummysource/template.param,dummysource/SkeletonApplication.template
generated:
    /home/zhen/work/pynb/sandbox/daq/bril/scripts/SkeletonApplication.xml
command per context:
   /opt/xdaq/bin/xdaq.exe -p 50000 -c /home/zhen/work/pynb/sandbox/daq/bril/scripts/SkeletonApplication.xml 

sources:
    dummysource/template.param,dummysource/LocalApplication_inzone.template
generated:
    /home/zhen/work/pynb/sandbox/daq/bril/scripts/LocalApplication_inzone.xml
command per context:
   /opt/xdaq/bin/xdaq.exe -p 50000 -c /home/zhen/work/pynb/sandbox/daq/bril/scripts/LocalApplication_inzone.xml -z bril

sources:
    dummysource/template.param,dummysource/LocalMonitora

In [16]:
# Prepare the command xdaqcmd to run in bash.
contextport=50000
xdaqcmd="/opt/xdaq/bin/xdaq.exe -p 50000 -c /home/zhen/work/pynb/sandbox/daq/bril/scripts/LocalMonitorable.xml"
mycmd="source ./brilenv.sh; "+xdaqcmd

In [17]:
# Run the command in background
# In bash, just run the above xdaqcmd with correct -p contextport and watch the screen
from subprocess import Popen , PIPE, STDOUT
proc=Popen(mycmd,stdin=PIPE,stdout=PIPE,stderr=PIPE,shell=True)

In [19]:
# Check if your application is running with hyperdaq 
# Note: in static html the xdaq iframe below will not display. That is fine
# In bash, open firefox and watch url http://hostname:contextport
# Icon brildummysource SkeletonApplication is clickable. On click, it shows the classname.
import socket
hostname = socket.gethostname()
xdaqcontexturl = 'http://%s:%d'%(hostname,contextport)
from IPython.display import IFrame
display(IFrame(xdaqcontexturl, '100%', '600px'))

In [20]:
# Click on icon "bril dummysource Monitorable"
# Every click on this icon (the application page) is a call to our method "Default".
# The pulled monitoring parameters and its web table is updated

In [21]:
# Stop SkeletonApplication 
# Ctrl-C is your process is running in the front
!./stopApp.sh 50000


                     USER        PID ACCESS COMMAND
50000/tcp:           zhen      31494 F.... xdaq.exe


##### check the pushed monitoring parameters 

In [22]:
# Run LocalMonitorableAndCollector or LocalMonitorableAndCollector_inzone 
# We load an additional bril::dummysource::MonitoringCollector in the context
# It attaches to the same monitoring infospaces as Monitorable, and it listens to ItemGroupChanged event.
#
# For less boredom, we changed the Monitorable to work in real mode.
# 
# Note: you are NOT required to write such collectors, there are central ones in zone that works as
# this simple collector in principle but in a much more elaborated implementation and setup.
# In the application, you just keep pushing. 
# If you are curious, the source code of the collector is in class MonitoringCollector
#
%cat $workdir/daq/bril/scripts/dummysource/LocalMonitorableAndCollector.template

<?xml version="1.0" encoding="UTF-8"?>
<xc:Partition xmlns:soapenc="http://schemas.xmlsoap.org/soap/encoding/" xmlns:xc="http://xdaq.web.cern.ch/xdaq/xsd/2004/XMLConfiguration-30" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">

 <xc:Context url="http://%hostname%:%contextport%">
    <xc:Endpoint protocol="utcp" service="b2in" rmode="select" hostname="%hostname%" port="%endpoint%" network="utcp1" sndTimeout="0" rcvTimeout="0" affinity="RCV:P,SND:W,DSR:W,DSS:W" singleThread="true" publish="true"/>

    <xc:Application class="pt::utcp::Application" id="12" instance="0" network="local">
      <properties xmlns="urn:xdaq-application:pt::utcp::Application" xsi:type="soapenc:Struct">
	<maxClients xsi:type="xsd:unsignedInt">10</maxClients>
        <autoConnect xsi:type="xsd:boolean">false</autoConnect>
        <ioQueueSize xsi:type="xsd:unsignedInt">65536</ioQueueSize>
        <eventQueueSize xsi:type="xsd:unsignedInt">65536</eventQueueSize>
        <maxReceiveBuffers xsi:type="xsd:uns

In [23]:
# Run suggested xdaq command with LocalMonitorableAndCollector or LocalMonitorableAndCollector_inzone 
# you should see message from MonitoringCollector about avgValue in regular interval
# avgValue starting from 0. then changed to 1.78250e+03